# Email 
___

A vessel's hold photos are sent to the vessel operator via email by either the vessel crew, inspector, or cleaning company.  The email often contains an excel file, a csv file, or attachments, and perhaps a cleaning report with planned cleaning, materials, etc.

The code here is being developped to:
* Receive an email containing photos of the cargo holds.
* Download the photos.
* Pass the photos to the model for analysis.
* Label the photos to further build the labeling directories.
* Return the model's analysis
* Provide cleaning recommendations.

In [72]:
import imaplib
import email
import os
import regex as re
import yaml

In [80]:
# function to retrieve credentials
def gmail_credentials(filepath):
    try:
        with open(filepath, 'r') as file:
            credentials = yaml.safe_load(file)
            user = credentials['user']
            password = credentials['app_password']
            return user, password
    except Exception as e:
        logging.error("Failed to load credentials: {}".format(e))
        raise

In [87]:
# function to login and return connection to mail server
def login_gmail(username, password):
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    mail.login(username, password)
    return mail

In [88]:
# connect to gmail
def connect_gmail():
    gmail = gmail_credentials('../gmail_credentials.yaml')
    connected = login_gmail(gmail[0], gmail[1])
    return connected

In [83]:
def clean_subject(subject):
    decoded_subject, encoding = decode_header(subject)[0]
    if isinstance(decoded_subject, bytes):
       decoded_subject = decoded_subject.decode(encoding if encoding else 'utf-8')
    return decoded_subject

In [84]:
def retrieve_new_emails(mail):
    # check inbox for new mail
    mail.select('inbox')
    status, messages = mail.search(None, 'UNSEEN')
    if status == 'OK':
        email_ids = messages[0].split()
        print("There are {len(messages[0].split())} new messages.")
        return email_ids
    else:
        print("No new messages found!")

In [85]:
def download_attachment(msg, download_folder="attachments"):
    # Check if the email message is multipart (contains attachments)
    if msg.is_multipart():
        for part in msg.walk():
            # If part is multipart, continue to next part
            if part.get_content_maintype() == 'multipart':
                continue
            # If part is an attachment
            if part.get('Content-Disposition') is not None:
                filename = part.get_filename()
                if filename:
                    # Decode filename if needed
                    filename, encoding = decode_header(filename)[0]
                    if isinstance(filename, bytes):
                        filename = filename.decode(encoding if encoding else 'utf-8')
                    # Save the attachment to the specified folder
                    if not os.path.exists(download_folder):
                        os.makedirs(download_folder)
                    filepath = os.path.join(download_folder, filename)
                    # Write attachment to the file
                    with open(filepath, "wb") as f:
                        f.write(part.get_payload(decode=True))
                    print(f"Attachment {filename} saved to {filepath}")

In [86]:
retrieve_new_emails(mail)

error: command SELECT illegal in state NONAUTH, only allowed in states AUTH, SELECTED

In [71]:
def main():
    # Connect to Gmail
    mail = connect_to_gmail()
    
    # Fetch emails
    email_ids = fetch_emails(mail)
    
    if not email_ids:
        print("No emails found.")
        return
    
    # Loop through all emails and process them
    for email_id in email_ids:
        status, msg_data = mail.fetch(email_id, "(RFC822)")  # Fetch the full email
        if status != "OK":
            print(f"Failed to fetch email ID {email_id}")
            continue

        # Get the email content
        for response_part in msg_data:
            if isinstance(response_part, tuple):
                msg = email.message_from_bytes(response_part[1])
                
                # Get the email subject
                subject = msg["subject"]
                subject = clean_subject(subject)
                print(f"Processing email: {subject}")
                
                # Download attachments from this email
                download_attachment(msg)
    
    # Logout from the server
    mail.logout()

if __name__ == "__main__":
    main()

NameError: name 'connect_to_gmail' is not defined

Sources

Masego - Medium
https://medium.com/@masego_m/accessing-gmail-with-python-a-beginners-guide-812e0068a568

Coding Point - YouTube
https://www.youtube.com/watch?v=HohkC2lNpZg

Tech with Time - YouTube
https://www.youtube.com/watch?v=rBEQL2tC2xY

ChatGPT - author, OpenAI - publisher, date=23 Nov 2024

In [57]:
def search_email():
    status, messages = mail.search(None, "ALL")
    email_ids = messages[0].split()
    for email_id in email_ids[:5]:  # Limit to first 5 emails
            status, msg_data = mail.fetch(email_id, "(RFC822)")
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    # Parse the email content
                    msg = email.message_from_bytes(response_part[1])
                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else "utf-8")
                    print(f"Subject: {subject}")
                    print(f"From: {msg.get('From')}")
                    print("-" * 50)


In [59]:
search_email()

error: command SEARCH illegal in state NONAUTH, only allowed in states SELECTED